In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import use
use('Agg')
import warnings
warnings.filterwarnings('ignore')
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
from scipy.optimize import curve_fit
from src.simulate_func import hill_func_new_mut

# Generate dose-response plot from Isunakinra variants in HEK Blue IL-1B cells

In [13]:
# Load simulation data 
df_sim = pd.read_csv("data/simulations.csv")
df_sim = df_sim[df_sim.columns[1:]]

In [14]:
# Load in vitro data
df_invit = pd.read_csv("data_IL1B/IL1B_data.csv")

In [15]:
# Directory location where to store plots
directory = "plots/"

In [16]:
# Colors for mutants:
color_sim = ["darkgreen","olivedrab","magenta","darkorange","blue"]
color_invit = ["forestgreen","yellowgreen","orchid","darkorange","cornflowerblue"]
markers = ['o','^',"P","X","s"]
linestyles = ["dotted","dashdot","dashed",(5, (10, 3)),(0, (3, 1, 1, 1, 1, 1))]

In [17]:
df_invit.loc[df_invit["Variant"]=="93:60","Variant"] = "93:60 (Isunakinra)"
df_sim.loc[df_sim["Variant"]=="93:60","Variant"] = "93:60 (Isunakinra)"

In [18]:
for plot_num in list(dict.fromkeys(df_sim["Plot"])):
    df_sim_plot = df_sim.loc[df_sim["Plot"]==plot_num]
    df_invit_plot = df_invit.loc[df_invit["Plot"]==plot_num]

    # First plot WT
    variant = list(dict.fromkeys(df_sim_plot["Variant"]))[0]
    fig,ax=plt.subplots(1,1,figsize=(9, 8), dpi=400)
    max_WT = df_sim_plot.loc[df_sim_plot["Variant"]==variant]["Result"].max()
    ax.scatter(df_invit_plot.loc[df_invit_plot["Variant"]==variant]["A"].values, df_invit_plot.loc[df_invit_plot["Variant"]==variant]["pSTAT"].values, s=100, linewidth=4, color="salmon",marker='*')
    ax.plot(df_sim_plot.loc[df_sim_plot["Variant"]==variant]["A0"].values, np.multiply(df_sim_plot.loc[df_sim_plot["Variant"]==variant]["Result"].values,100/(max_WT)), linewidth=7.5, label=variant,color="darkred", linestyle="solid")
    i=0
    # Then plot the other variants
    for variant in list(dict.fromkeys(df_sim_plot["Variant"]))[1:]:
        ax.scatter(df_invit_plot.loc[df_invit_plot["Variant"]==variant]["A"].values, df_invit_plot.loc[df_invit_plot["Variant"]==variant]["pSTAT"].values, s=100, linewidth=4, color=color_invit[i],marker=markers[i])
        ax.plot(df_sim_plot.loc[df_sim_plot["Variant"]==variant]["A0"].values, np.multiply(df_sim_plot.loc[df_sim_plot["Variant"]==variant]["Result"].values,100/(max_WT)), linewidth=7.5, label=variant,color=color_sim[i], linestyle=linestyles[i])
        i += 1
    plt.legend(loc="upper right", fontsize=15)
    ax.set_xticks(10**np.arange(round(np.log10(df_sim_plot["A0"].min()),0)+1,round(np.log10(df_sim_plot["A0"].max()),0)+1,2))
    ax.spines["bottom"].set_linewidth(4)
    ax.spines["left"].set_linewidth(4)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.xaxis.set_tick_params(width=5, length=10)
    ax.yaxis.set_tick_params(width=5, length=10)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    plt.xscale('log')
    plt.ylabel('IL1B signaling (%)', fontsize=25)
    plt.xlabel('Antagonist (M)', fontsize=25)
    plt.savefig(directory +'Plot_n'+str(df_sim_plot["Plot"].values[0])+'_cell_'+df_sim_plot["Cell_type"].values[0]+'.png', transparent=True, bbox_inches="tight")
    plt.savefig(directory +'Plot_n'+str(df_sim_plot["Plot"].values[0])+'_cell_'+df_sim_plot["Cell_type"].values[0]+'.svg', transparent=True, bbox_inches="tight", format="svg")
    plt.show()

# Generate heat map of diffences in IC50

In [2]:
df_heat = pd.read_csv("data/heat_map_isu.csv")

In [8]:
# Take data
log_y_vec = np.log10(list(dict.fromkeys(df_heat["IL"].values)))
log_x_vec = list(dict.fromkeys(df_heat["R1"].values))
dEC50_mat = []
for IL_val in list(dict.fromkeys(df_heat["IL"].values)):
    df_IL = df_heat.loc[df_heat["IL"] == IL_val]
    dEC50_mat.append(df_IL["dEC50"].tolist())
dEC50_mat = np.array(dEC50_mat)

In [ ]:
# Save data in plots
log_x, log_y = np.meshgrid(log_x_vec, log_y_vec)
fig,ax=plt.subplots(1,1,figsize=(8, 6), dpi=600)
cf = ax.pcolormesh(log_x_vec, log_y_vec, dEC50_mat, cmap='viridis', shading='nearest')
plt.scatter(3.647365869373922/1.2, np.log10(6e-12), color="red", linewidth=6)
ax.text(3.647365869373922/1.2+0.025, np.log10(6e-12)+0.2, "HEK",fontsize=16, color="red")
plt.scatter(3.202618857361186/1.2, np.log10(6e-12), color="green", linewidth=6)
ax.text(3.202618857361186/1.2-0.5, np.log10(6e-12)+0.2, "HT29",fontsize=16, color="green")
ax.set_ylabel("log10 [IL1B] (M)", fontsize=16,labelpad=7)
ax.set_xlabel("log10 IL1R1 expression", fontsize=16,labelpad=7)
cb = fig.colorbar(cf)
cb.set_label(label="log10 dIC50 (M)", size=16)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)
plt.savefig('plots/dIC50_heat_map_isu.png', transparent=True, bbox_inches="tight")
plt.close(fig)

# Instead of heat map, plot only one IL1B concentration (6*10⁻¹²)

In [9]:
def hill_func_heat_map(inp,K,amp,n):
    out=amp/(1+(inp/K)**n)
    return out

In [10]:
df_heat = pd.read_csv("data/heat_map_isu.csv")
R1_expr = df_heat.loc[df_heat["IL"] == 6.000000000000003e-12,"R1"].values
dIC50 = df_heat.loc[df_heat["IL"] == 6.000000000000003e-12,"dEC50"].values
fit, cov = curve_fit(hill_func_heat_map, R1_expr, dIC50, bounds = ([10.5,1,4], [15,1.1,5]))

In [11]:
R1_expr_curve=np.linspace(2**R1_expr.min(),2**R1_expr.max(),30)

In [12]:
fig,ax=plt.subplots(1,1,figsize=(9, 8), dpi=400)
ax.plot(3.763891690163789/1.2*np.ones(2), [dIC50.min(),dIC50.max()], color="blue", linewidth=5, linestyle="dashed", zorder=1) 
ax.text(3.763891690163789/1.2+0.025, 0.1, "HEK",fontsize=20, color="blue", zorder=2)
ax.plot(3.202618857361186/1.2*np.ones(2), [dIC50.min(),dIC50.max()], color="darkgreen", linewidth=5, linestyle="dashed", zorder=3) 
ax.text(3.202618857361186/1.2-0.5, 0.1, "HT29",fontsize=20, color="darkgreen", zorder=4)
ax.scatter(R1_expr, dIC50, s=100, linewidth=4, color="salmon",marker='*', zorder=5)
ax.plot(np.log2(R1_expr_curve), hill_func_heat_map(R1_expr_curve,fit[0],fit[1],fit[2]), linewidth=7.5,color="darkred", linestyle="solid", zorder=6)
ax.spines["bottom"].set_linewidth(4)
ax.spines["left"].set_linewidth(4)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.xaxis.set_tick_params(width=5, length=10)
ax.yaxis.set_tick_params(width=5, length=10)
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
plt.xlabel("log10 IL1R1 expression", fontsize=25)
plt.ylabel("log10 dIC50 (M)", fontsize=25)
plt.savefig('plots/dIC50_single_concentration.png', transparent=True, bbox_inches="tight")
plt.savefig('plots/dIC50_single_concentration.svg', transparent=True, bbox_inches="tight", format="svg")
plt.close(fig)

# Get predicetd IC50 ratio of new mutant/WT in HEK/HT29 cells

In [3]:
# Load simulation data 
df_sim = pd.read_csv("data/simulations_New_mut.csv")
df_sim = df_sim[df_sim.columns[1:]]

df_HEK = df_sim.loc[df_sim["Cell_type"]=="ACH-000049"]
df_HT29 = df_sim.loc[df_sim["Cell_type"]=="ACH-000552"]

df_var = df_HEK.loc[df_HEK["Variant"] == "93:60"]
max_WT = df_var["Result"].max()
fit, cov = curve_fit(hill_func_new_mut, df_var["A0"], df_var["Result"].values*100/(max_WT), bounds = ([df_var["A0"].min()/100,max_WT-17], [df_var["A0"].max()*100,max_WT+17]))
Isun_HEK_IC50 = fit[0]

df_var = df_HEK.loc[df_HEK["Variant"] == "Mut"]
max_WT = df_var["Result"].max()
fit, cov = curve_fit(hill_func_new_mut, df_var["A0"], df_var["Result"].values*100/(max_WT), bounds = ([df_var["A0"].min()/100,max_WT-17], [df_var["A0"].max()*100,max_WT+17]))
Mut_HEK_IC50 = fit[0]

print("Experimental ratio WT/Mutant (HEK): " + str(6.5))
print("Modeled ratio WT/Mutant (HEK): " + str(Isun_HEK_IC50/Mut_HEK_IC50))

df_var = df_HT29.loc[df_HT29["Variant"] == "93:60"]
max_WT = df_var["Result"].max()
fit, cov = curve_fit(hill_func_new_mut, df_var["A0"], df_var["Result"].values*100/(max_WT), bounds = ([df_var["A0"].min()/100,max_WT-17], [df_var["A0"].max()*100,max_WT+17]))
Isun_HT29_IC50 = fit[0]

df_var = df_HT29.loc[df_HT29["Variant"] == "Mut"]
max_WT = df_var["Result"].max()
fit, cov = curve_fit(hill_func_new_mut, df_var["A0"], df_var["Result"].values*100/(max_WT), bounds = ([df_var["A0"].min()/100,max_WT-17], [df_var["A0"].max()*100,max_WT+17]))
Mut_HT29_IC50 = fit[0]

print("Experimental ratio WT/Mutant (HT29): " + str(22))
print("Modeled ratio WT/Mutant (HT29): " + str(Isun_HT29_IC50/Mut_HT29_IC50))

Experimental ratio WT/Mutant (HEK): 6.5
Modeled ratio WT/Mutant (HEK): 1.2414258807797065
Experimental ratio WT/Mutant (HT29): 22
Modeled ratio WT/Mutant (HT29): 13.371562549279778


In [13]:
df_bind = pd.read_csv('data_IL1B/IL1B_data_param_fit.csv')
# Chnage df_bind to add the new mutant to fit
df_bind = df_bind.loc[[5]][df_bind.columns[1:]]
df_bind = df_bind.reset_index(drop=True)
df_bind.loc[1] = df_bind.loc[0]
df_bind.loc[1,"Variant"] = "Mut"
df_bind.loc[1,"k_A_R1_f"] = 3794328.37281239

print("Model predicts mutant has a binding rate "+str(round(df_bind["k_A_R1_f"].values[1]/df_bind["k_A_R1_f"].values[0],2))+" times higher")

Model predicts mutant has a binding rate 6.0 times higher
